In [1]:
from web3 import Web3
from web3.eth import Eth
from eth_account import Account
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256, SHA512
import struct
import base64
import hmac
import hashlib


In [2]:
BS = 16
def pad(data):
    padding = BS - len(data) % BS
    return data + padding * struct.pack("B", padding)

def unpad(data):
    return data[0:-data[-1]]

def AES256CbcDecrypt(hex_data, key=b'0'*32, iv=b'0'*16):
    data = bytearray.fromhex(hex_data)
    aes = AES.new(key, AES.MODE_CBC, iv)
    return unpad(aes.decrypt(data))

def AES256CbcEncrypt(bin_data, key=b'0'*32, iv=b'0'*16):
    aes = AES.new(key, AES.MODE_CBC, iv)
    return aes.encrypt(pad(bin_data))

In [3]:
#Test pading /unpading
start = bytearray(b'test')
padded = pad(start)
print(len(padded))
unpaded = unpad(padded)
print (unpaded)
print (unpaded==start)

16
bytearray(b'test')
True


In [4]:
#Test encrypt
hex_key = '1ac27dc3580fa322d795d54fe4b4b211980a299f924f1df267195c4f8253b700'
key = bytearray.fromhex(hex_key)
text ='test'
plaintext = text.encode(encoding='utf_8') 
hex_iv='95eb6bfc4ddcd00ba1b2d63b7c26bff4'
iv= bytearray.fromhex(hex_iv)

cry = AES256CbcEncrypt(plaintext,key,iv).hex()
print(cry)
print(cry=='e4bec3c272130e40fa34f068819d8e83')

#test decrypt
dec = AES256CbcDecrypt(cry,key,iv)
print(dec.decode("utf-8"))
print(dec.decode("utf-8")==text)

e4bec3c272130e40fa34f068819d8e83
True
test
True


In [5]:
def encode_key(pub_key):
    return pub_key.pointQ.x.to_bytes(32)+ pub_key.pointQ.y.to_bytes(32)

In [6]:
test_encoded = (74112620180903413835928205978618879887273296202846370176989995621597768248548).to_bytes(32,byteorder='big')+(70352546531620714376440768798610125065500941604960487601320179994127409968403).to_bytes(32,byteorder='big')
[ test_encoded[i] for i in range(len(test_encoded)) ] == [163, 218, 65, 170, 47, 136, 127, 159, 98, 77, 71, 242, 112, 244, 229, 36, 177, 2, 147, 111, 8, 233, 154, 255, 238, 244, 221, 88, 15, 2, 236, 228, 155, 138, 33, 87, 29, 185, 26, 29, 154, 176, 239, 200, 165, 137, 137, 181, 96, 137, 184, 159, 197, 133, 182, 251, 181, 179, 184, 113, 179, 141, 117, 19]

True

In [7]:
def decode_key(encoded_pub_key):
    x = encoded_pub_key[:32]
    y = encoded_pub_key[32:]
    return ECC.EccPoint(int.from_bytes(x,byteorder='big'), int.from_bytes(y,byteorder='big'))

In [8]:
ephemPrivKey = ECC.generate(curve='secp256r1')
encoded = encode_key(ephemPrivKey.public_key())
decode_key(encoded)

In [9]:
def derive(private_key, pub_ecc_point):
    return (pub_ecc_point*private_key.d).x

In [11]:
mac = hmac.new(b'key',b'test', hashlib.sha256).digest()
print([ mac[i] for i in range(len(mac)) ] == [2, 175, 181, 99, 4, 144, 44, 101, 111, 203, 115, 124, 221, 3, 222, 98, 5, 187, 109, 64, 29, 162, 129, 46, 253, 155, 45, 54, 160, 138, 241, 89])

True


In [12]:
mackey = bytearray([79, 239, 151, 70, 153, 199, 149, 191, 63, 138, 31, 222, 246, 48, 101, 30, 50, 242, 204, 220, 2, 110, 67, 183, 43, 138, 60, 2, 197, 100, 91, 198])
dataToMac=bytearray([149, 235, 107, 252, 77, 220, 208, 11, 161, 178, 214, 59, 124, 38, 191, 244, 4, 75, 110, 138, 127, 216, 95, 226, 105, 190, 209, 196, 201, 218, 135, 24, 64, 123, 111, 244, 29, 172, 190, 228, 139, 187, 179, 195, 2, 222, 119, 128, 51, 141, 128, 187, 166, 9, 238, 207, 172, 236, 18, 138, 135, 185, 9, 245, 30, 102, 165, 121, 224, 29, 40, 205, 213, 202, 79, 101, 78, 96, 28, 198, 185, 228, 190, 195, 194, 114, 19, 14, 64, 250, 52, 240, 104, 129, 157, 142, 131])
mac = hmac.new(mackey,dataToMac, hashlib.sha256).digest()
[ mac[i] for i in range(len(mac)) ] ==  [14, 120, 220, 2, 214, 66, 118, 74, 3, 156, 174, 142, 155, 38, 96, 183, 178, 132, 74, 102, 3, 195, 47, 32, 45, 59, 109, 239, 77, 155, 27, 130]

True

---------------------------------

In [63]:
test_key_priv = ECC.generate(curve='secp256r1')
test_key_pub = test_key_priv.public_key()

test_key_priv_hex = hex(int(test_key_priv.d))
test_key_pub_hex = hex(int(test_key_pub.pointQ.x)) + hex(int(test_key_pub.pointQ.y))[2:]

print(test_key_priv_hex)
print(test_key_pub_hex)

0x4468a5720b9424da6173c821f29e7645e550c01e1aa3c1a8bbe700ef7829f0d
0xfa6e5e5e6b2fde22decb9905a65186a63e95151ccfc78894f9a91279312bf1ac2e6352727c6c81ae85b93c46206a0cea41d1f7f21050488528672eb858ae1b12


In [64]:
def EncryptMessage(publicKey_hex, plainText_string):
    if publicKey_hex[:2]=='0x':
        publicKey_hex=publicKey_hex[2:]
    
    # Generate the temporary key 
    ephemPrivKey = ECC.generate(curve='secp256r1')
    ephemPubKey = ephemPrivKey.public_key()
    ephemPubKeyEncoded = encode_key(ephemPubKey)
    
    # Load the public key
    publicKey = bytearray.fromhex(publicKey_hex)
    pub = ECC.EccPoint(int.from_bytes(publicKey[:int(len(publicKey)/2)],byteorder='big'), int.from_bytes(publicKey[int(len(publicKey)/2):],byteorder='big'))
    
    # ECDH => get the shared secret
    px = derive(ephemPrivKey,pub) 
    
    # compute the encription and MAC keys
    hash_px = SHA512.new(data=px.to_bytes()).digest()
    encryptionKey = hash_px[:32]
    macKey = hash_px[32:]
    
    # cipher the plain text
    iv = Random.get_random_bytes(16)
    plaintext = plainText_string.encode(encoding='utf_8') 
    ciphertext = AES256CbcEncrypt(plaintext,encryptionKey,iv)
    
    # compute the MAC
    dataToMac = iv + ephemPubKeyEncoded + ciphertext
    mac = hmac.new(macKey, dataToMac, 'sha256').digest()
    
    #build the output
    serializedCiphertext = iv + ephemPubKeyEncoded + mac + ciphertext
    return serializedCiphertext.hex()
    

In [65]:
message = 'test'
encrypted = EncryptMessage(test_key_pub_hex, message)
print(encrypted)

43594997613cd5d96f728c43ac289e4f920173475037c93d8d370b74912d221443dc5899fd5af90d4dc245a8b7fd8ffe86b67f8f1c502b1ac400bd73905d8e7494b74042fdadc0dfda16c34fab14887ed604f40fe80903ed2a0667f1388e9f96e08fbbb8a2ded25feb1cc3ff5b89e527b00081421cc32a015f8eaab86b2f2193


In [66]:
def DecryptMessage(privateKey_hey, encrypted_hex):
    if privateKey_hey[:2]=='0x':
        privateKey_hey=privateKey_hey[2:]
    if encrypted_hex[:2]=='0x':
        encrypted_hex=encrypted_hex[2:]

    # get the components 
    encrypted = bytearray.fromhex(encrypted_hex)
    iv = encrypted[:16]
    ephemPubKeyEncoded = encrypted[16:80]
    mac = encrypted[80:112]
    ciphertext = encrypted[112:]
    
    # recover the temporary public key
    ephemPubKey = decode_key(ephemPubKeyEncoded)
    
    # load the private key
    privKey = ECC.construct(curve='secp256r1', d=int(privateKey_hey,16))
    
    # ECDH => get the shared secret
    px = derive(privKey, ephemPubKey)
    
    # compute the encription and MAC keys
    hash_px = SHA512.new(data=px.to_bytes()).digest()
    encryptionKey = hash_px[:32]
    macKey = hash_px[32:]
    
    # check the MAC
    dataToMac = iv + ephemPubKeyEncoded + ciphertext
    computed_mac = hmac.new(macKey, dataToMac, 'sha256').digest()
    if computed_mac!=mac:
        raise ValueError("MAC missmatch")
        
    #decipher the text
    plaintext = AES256CbcDecrypt(ciphertext.hex(), encryptionKey, iv)
    return plaintext.decode("utf-8")
    

In [68]:
message = 'test'
encrypted = EncryptMessage(test_key_pub_hex, message)


mm = DecryptMessage(test_key_priv_hex, encrypted)
print(mm)

test
